In [ ]:
from bs4 import BeautifulSoup as bs

In [ ]:
import pandas as pd 

In [ ]:
import json
import requests
from tqdm.notebook import tqdm

In [ ]:
import re

In [ ]:
url = 'https://www.goodchoice.kr/product/recommend/153'

resp = requests.get(url)
soup = bs(resp.content, 'lxml')

In [ ]:
#메인 검색에서 url_list 가저옴 
url_list = []
url = soup.select('div.list_wrap a')


for i in url:
    url_list.append(i['href'])

In [ ]:
len(url_list)

500

In [ ]:
#메인 페이지에서 title_list 가져옴 
# 500개로 같은거 확인 
title_list = [] 
title = soup.select('div.name strong')
for i in title:
    print(i.text.strip())
    title_list.append(i.text.strip())


In [ ]:
len(title_list)

500

In [ ]:
#페이로드에 존재하는 상품명까지 가져오기 .
product_num = []
for url in url_list:
    product_num.append(re.findall('ano=(.+)\&ad',url))

In [ ]:
len(product_num)

500

In [ ]:
#페이지 내에서 리뷰 + 별점 + 구성용품 가저 오기 
url = 'https://www.goodchoice.kr/product/get_review_non' 

headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'referer': 'https://www.goodchoice.kr/product/detail?ano=66407&adcno=5&sel_date=2023-02-06&sel_date2=2023-02-07'
}

data = {
    'page': '0',
    'ano': '71091'
}

resp = requests.post(url,data = data ,headers = headers)

resp.text




'\n{"result":{"ratehouse":0,"rateprice":0,"rateservice":0,"rateavg":0,"ownercommcnt":0,"rate_textinfo":"","userstate":"N","total_page_cnt":0,"items":[],"count":0,"score_star":"00"},"total_page":0}'

In [ ]:
resp.content.lstrip()

b'{"result":{"ratehouse":0,"rateprice":0,"rateservice":0,"rateavg":0,"ownercommcnt":0,"rate_textinfo":"","userstate":"N","total_page_cnt":0,"items":[],"count":0,"score_star":"00"},"total_page":0}'

In [ ]:
result = json.loads(resp.content.lstrip())
result

{'result': {'ratehouse': 0,
  'rateprice': 0,
  'rateservice': 0,
  'rateavg': 0,
  'ownercommcnt': 0,
  'rate_textinfo': '',
  'userstate': 'N',
  'total_page_cnt': 0,
  'items': [],
  'count': 0,
  'score_star': '00'},
 'total_page': 0}

In [ ]:
result['result']['rateavg']

###별점 get !  

0

In [ ]:
comment = [] 
for i in result['result']['items']:
    comment.append(i['aepcont'])

if len(comment) == 0 :
    comment.append("리뷰가 없습니다")

In [ ]:
comment

['리뷰가 없습니다']

In [ ]:
#부대시설 가져오기 

url1 = 'https://www.goodchoice.kr/product/get_theme_list_non' 

headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'referer': 'https://www.goodchoice.kr/product/detail?ano=66407&adcno=5&sel_date=2023-02-06&sel_date2=2023-02-07'
}
data1 = {
    'ano': '66407'
}

resp1 = requests.post(url1,data = data1 ,headers = headers)

resp1.text


'\n{"code":200,"data":[{"tmino":81,"tminame":"\\uac1c\\uc218\\ub300","images":[]},{"tmino":85,"tminame":"\\ub9e4\\uc810","images":[]},{"tmino":270,"tminame":"\\uc8fc\\ucc28\\uc7a5","images":[]},{"tmino":273,"tminame":"BBQ","images":[]},{"tmino":276,"tminame":"\\uc640\\uc774\\ud30c\\uc774","images":[]},{"tmino":277,"tminame":"TV","images":[]},{"tmino":278,"tminame":"\\uc695\\uc2e4\\uc6a9\\ud488","images":[]},{"tmino":281,"tminame":"\\uc5d0\\uc5b4\\ucee8","images":[]},{"tmino":282,"tminame":"\\ub0c9\\uc7a5\\uace0","images":[]},{"tmino":283,"tminame":"\\uac1d\\uc2e4\\uc0e4\\uc6cc\\uc2e4","images":[]},{"tmino":285,"tminame":"\\ub4dc\\ub77c\\uc774\\uae30","images":[]},{"tmino":287,"tminame":"\\uc804\\uae30\\ubc25\\uc1a5","images":[]},{"tmino":291,"tminame":"\\ubc18\\ub824\\uacac\\ub3d9\\ubc18","images":[]},{"tmino":292,"tminame":"\\uac1d\\uc2e4\\ub0b4\\ucde8\\uc0ac","images":[]},{"tmino":321,"tminame":"\\uae08\\uc5f0","images":[]}],"msg":"OK","querytime":0,"servertime":1675648467}'

In [ ]:
facility = []
result1 = json.loads(resp1.content.lstrip())
for i in result1['data']:
    facility.append(i['tminame'])

facility

['개수대',
 '매점',
 '주차장',
 'BBQ',
 '와이파이',
 'TV',
 '욕실용품',
 '에어컨',
 '냉장고',
 '객실샤워실',
 '드라이기',
 '전기밥솥',
 '반려견동반',
 '객실내취사',
 '금연']

In [ ]:
product_num

In [ ]:
rate = []
total_comments = []
total_facility = [] 
review_count = []
for num in tqdm(product_num):
    comment = [] 
    facility = []

    #페이지 내에서 리뷰 + 별점 + 구성용품 가저 오기 
    url = 'https://www.goodchoice.kr/product/get_review_non' 

    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
        'referer': 'https://www.goodchoice.kr/product/detail?ano=66407&adcno=5&sel_date=2023-02-06&sel_date2=2023-02-07'
    }

    data = {
        'page': '0',
        'ano': num
    }

    resp = requests.post(url,data = data ,headers = headers)

    resp.text
    resp.content.lstrip()

    result = json.loads(resp.content.lstrip())
    rate.append(result['result']['rateavg'])

    #각 호텔별 리뷰 가져오기 
    for i in result['result']['items']:
        comment.append(i['aepcont'])
        if len(comment) == 0 :
            comment.append("리뷰가 없습니다")
    #전체에 공유할 하나의 호텔 리뷰들 저장
    total_comments.append(comment)
    #리뷰 개수 저장 
    review_count.append(len(comment))

    for i in result1['data']:
        facility.append(i['tminame'])
    total_facility.append(facility)
    

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
len(total_comments),len(total_facility), len(review_count), len(rate) 

(500, 500, 500, 500)

In [ ]:
import pandas as pd 

df = pd.DataFrame({'name' : title_list, 'num' : product_num, 'url': url_list, 'rate': rate, 'review_num': review_count,  'comments' : total_comments, 'facility': total_facility})
df

,name,num,url,rate,review_num,comments,facility
0,포천 산울터애견동반펜션,[61224],https://www.goodchoice.kr/product/detail?ano=6...,9.0,4,"[부모님 모시고 갔는데 너무 좋았습니다, 급하게 예약했는데 가격이 저렴하지만 전부 ...","[개수대, 매점, 주차장, BBQ, 와이파이, TV, 욕실용품, 에어컨, 냉장고, ..."
1,고성 설악의정원 애견펜션&카페,[67715],https://www.goodchoice.kr/product/detail?ano=6...,10.0,7,[다른데는 강아지 종 따지고 소형견만 된다고 했는데\n15~16키로 나가는 중형견 ...,"[개수대, 매점, 주차장, BBQ, 와이파이, TV, 욕실용품, 에어컨, 냉장고, ..."
2,안동 하회로가는 애견동반펜션,[68156],https://www.goodchoice.kr/product/detail?ano=6...,10.0,5,[부모님께서 그간 여러군데 펜션을 다녀봤지만 이렇게 정있고 세심하게 챙겨주는 곳은 ...,"[개수대, 매점, 주차장, BBQ, 와이파이, TV, 욕실용품, 에어컨, 냉장고, ..."
3,거제 여울애견펜션,[67079],https://www.goodchoice.kr/product/detail?ano=6...,10.0,2,[여울팬션 완젼 쵝오입니다!!\n사장님께서 직접 손으로 적어주신 주변 ...,"[개수대, 매점, 주차장, BBQ, 와이파이, TV, 욕실용품, 에어컨, 냉장고, ..."
4,영흥도 아침해변펜션,[71612],https://www.goodchoice.kr/product/detail?ano=7...,0.0,0,[],"[개수대, 매점, 주차장, BBQ, 와이파이, TV, 욕실용품, 에어컨, 냉장고, ..."
...,...,...,...,...,...,...,...
495,경주 하녹풀빌라,[70461],https://www.goodchoice.kr/product/detail?ano=7...,0.0,0,[],"[개수대, 매점, 주차장, BBQ, 와이파이, TV, 욕실용품, 에어컨, 냉장고, ..."
496,넓은 마당에서 반려견과 함께 힐링할 수 있는 숙소 | 구름산코지하우스,[72982],https://www.goodchoice.kr/product/detail?ano=7...,0.0,0,[],"[개수대, 매점, 주차장, BBQ, 와이파이, TV, 욕실용품, 에어컨, 냉장고, ..."
497,동해 망상파란펜션,[57511],https://www.goodchoice.kr/product/detail?ano=5...,7.1,15,[다시는 가고싶지않은숙소 입니다~\n204호 2인기준 최대4인 이라했지만 넷이서 쓰...,"[개수대, 매점, 주차장, BBQ, 와이파이, TV, 욕실용품, 에어컨, 냉장고, ..."
498,김천 오색애견펜션,[67908],https://www.goodchoice.kr/product/detail?ano=6...,9.9,12,"[얘견동반 추천드립니다^^ 시설도 깨끗하고 사장님도 친절하십니다, 바로 앞에 편의점...","[개수대, 매점, 주차장, BBQ, 와이파이, TV, 욕실용품, 에어컨, 냉장고, ..."


In [ ]:
df.to_csv("어기어때_결과(총3487개).csv", index = False)

In [ ]:
comments_len = 0 
for i in total_comments:
    comments_len += len(i)

comments_len

3487